# Packages 

In [1]:
#setup packages
from selenium import webdriver #v4.5.0
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time, urllib.request
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
import pyperclip
import pandas as pd
from selenium.webdriver.common.by import By
import linecache
import csv
import random

# Functions

In [2]:
def login(loginUsername, loginPassword):
    #navigate homepage to login 
    time.sleep(random.randint(5, 10))
    username=driver.find_element(By.CSS_SELECTOR, "input[name='username']")
    password=driver.find_element(By.CSS_SELECTOR, "input[name='password']")
    username.clear()
    password.clear()
    username.send_keys(loginUsername)
    time.sleep(random.randint(5, 10))
    password.send_keys(loginPassword)
    time.sleep(random.randint(1, 5))
    login = driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
    
    #save your login info
    time.sleep(random.randint(10, 17))
    notnow = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()
    #turn on notif
    time.sleep(random.randint(13, 15))
    notnow2 = driver.find_element(By.XPATH, "//button[contains(text(), 'Not Now')]").click()

In [3]:
def search(accountName):
    #navigate to instagram page for account
    time.sleep(random.randint(3, 7))
    driver.get(f'https://www.instagram.com/{accountName}/')

In [4]:
def navigateFollowerList(accountName):
    time.sleep(random.randint(3, 7))
    followersNum = getFollowerCount()
    
    #Select Followers Button
    driver.get(f'https://www.instagram.com/{accountName}/followers/')
    
    #Infinite Scoll Pop-Up
    time.sleep(random.randint(3, 7))
    scr1 = driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div[2]/div/div/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[2]")
    t_end = time.time() + 60 * followersNum/200 # 200 followers per minute, 650 was too fast
    while time.time() < t_end:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scr1)
    time.sleep(3)

In [5]:
def getFollowerList():
    #CTRL A to select all text
    ActionChains(driver)\
        .key_down(Keys.CONTROL)\
        .send_keys("a")\
        .perform()
    
    #CTRL C to copy all text
    ActionChains(driver)\
        .key_down(Keys.CONTROL)\
        .send_keys("c")\
        .perform()
    
    # copy followers
    s = pyperclip.paste()
    return s

In [6]:
def getFollowerCount():
    time.sleep(random.randint(3, 7))
    followerCount = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[1]/div/div/div/div[1]/div[1]/div[2]/section/main/div/header/section/ul/li[2]/a/div/span/span").text
    followerCount = int(followerCount.replace(',', ''))
    return followerCount

In [7]:
def getFollowingCount():
    time.sleep(random.randint(3, 7))
    followingCount = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[1]/div/div/div/div[1]/div[1]/div[2]/section/main/div/header/section/ul/li[3]/a/div/span/span").text
    followingCount = int(followingCount.replace(',', ''))
    return followingCount

In [16]:
def parseText(accountName):
    # delete blank lines
    with open(f'raw/{accountName}_rawFile.txt','r', encoding="utf-8") as infile, open(f'raw/{accountName}_deletedLines.txt','w', encoding="utf-8") as outfile:
        for line in infile:
            if not line.strip(): continue
            outfile.write(line)
    
    #close files
    infile.close()
    outfile.close()
    
    #open raw data with lines deleted
    rawFile = open(f'raw/{accountName}_deletedLines.txt','r', encoding="utf-8")
            
    #line counter
    lineCount = 0

    #determine the line num for followers
    while True:
        lineCount += 1
        line = rawFile.readline()
        if line.strip() == "Followers":
            rawFile.close()
            break
        if not line:
            print("Followers not Found.")
            rawFile.close()
            break

    # create file of just follower names
    followerFile = open(f'follower/{accountName}_followerFile.txt','w', encoding="utf-8")
    while True:
        line = linecache.getline(f'raw/{accountName}_deletedLines.txt', lineCount)
    
        # grab follower names and write to new file
        if "profile picture" in line:
            line = line.split("'")[0]
            followerFile.write("{}\n".format(line))
        
        if not line:
            #print("Parsing Complete.")
            followerFile.close()
            break
    
        lineCount += 1

    #close files
    rawFile.close()
    followerFile.close()

In [9]:
def setupCSV():
    csvFile = open('edges.csv','w',encoding="utf-8")
    csvWriter = csv.writer(csvFile)
    
    #create header
    csvWriter.writerow(['target','source','type','target_followers','target_following'])
    
    csvFile.close()

In [23]:
def updateCSV(accountName, targetFollowers, targetFollowing):
    #open followers
    followerFile = open(f'follower/{accountName}_followerFile.txt','r', encoding="utf-8")
    Lines = followerFile.readlines()
    
    for line in Lines:
        rowUpdate(accountName, line.strip(), targetFollowers, targetFollowing)
    
    #close file
    followerFile.close()

In [11]:
def rowUpdate(accountName, sourceName, targetFollowers, targetFollowing):
    # row data
    rowData = [accountName, sourceName, 'directed', targetFollowers, targetFollowing]
    
    # write data
    with open('edges.csv','a', encoding="utf-8") as csvFile:
        csvWriter = csv.writer(csvFile)
        csvWriter.writerow(rowData)
        csvFile.close()

In [12]:
def getFollowers(followerAccount):
    # navigate to page 
    search(followerAccount)
    
    # check if private
    time.sleep(random.randint(3, 8))
    try:
        target = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div[1]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div/article/div/div/h2")
        isPublic = False
        print("Account is private")
    except:
        isPublic = True
        print("Account is public")
    
    # build follower list
    if isPublic == True:
        # get target followers/following
        targetFollowers = getFollowerCount()
        targetFollowing = getFollowingCount()
        
        # navigate follower list
        navigateFollowerList(followerAccount)
        
        # create text file
        subFollowerList = getFollowerList()
        with open(f'raw/{followerAccount}_rawFile.txt','w', encoding="utf-8") as g:
            g.write(subFollowerList)
            g.close()
        
        #parse
        parseText(followerAccount)
        
        #update CSV
        updateCSV(followerAccount, targetFollowers, targetFollowing)

# Main Code

In [24]:
# launch chrome and navigate to instagram
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.instagram.com/")

# login into instagram, fill in username/password as needed
login("username","password!") #username, password

# search and navigate to account
accountName = "accountName" #initial account name here
search(accountName)

# get target followers/following
targetFollowers = getFollowerCount()
targetFollowing = getFollowingCount()

# navigate followers
navigateFollowerList(accountName)

# create text file
followerList = getFollowerList()
with open(f'raw/{accountName}_rawFile.txt','w', encoding="utf-8") as g:
    g.write(followerList)
    g.close()
    
# parse text file 
parseText(accountName)

# setup CSV file
setupCSV()
updateCSV(accountName, targetFollowers, targetFollowing)

## Create a dataframe 
edgesDataFrame = pd.read_csv("edges.csv")
accountNameCounter = 0

#Do again for followers of central
while accountNameCounter <= edgesDataFrame.shape[0]:
    accountName = edgesDataFrame['source'][accountNameCounter]
    print(accountName)
    time.sleep(random.randint(60, 140))
    getFollowers(accountName)
    accountNameCounter += 1
    
#do recursivley for all accounts
    #create list of used accounts
    #search CSV first columun until we get an account not in the list
    # repeat above

srijaupec596870
Account is public
adamkososki
Account is public
bbrambo


KeyboardInterrupt: 